# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [23]:
#imports 
import re
import functools
from collections import Counter, defaultdict
import numpy as np
import random

In [24]:
# Your code here

class ContextSensitiveSpellCorrector:
    def __init__(self, corpus_path, bigram_path=None):
        # Load unigram corpus and build word probability distribution.
        self.unigram_counts = self.load_corpus(corpus_path)
        self.total_words = sum(self.unigram_counts.values())
        self.word_probs = {w: c / self.total_words for w, c in self.unigram_counts.items()}
        
        # Load bigram frequencies if provided; expected format: count word1 word2
        if bigram_path:
            self.bigram_counts = self.load_bigrams(bigram_path)
            # Precompute totals for each preceding word to avoid repeated summing.
            self.bigram_totals = defaultdict(int)
            for (prev, word), count in self.bigram_counts.items():
                self.bigram_totals[prev] += count
        else:
            self.bigram_counts = None

    def load_corpus(self, path):
        """Load the text corpus and count word frequencies."""
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read().lower()
        pattern = re.compile(r'\w+')
        words = pattern.findall(text)
        return Counter(words)
    
    def load_bigrams(self, path):
        """Load bigram counts from a file."""
        bigrams = {}
        with open(path, 'r', encoding='latin-1', errors='ignore') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 3:
                    count, w1, w2 = parts
                    bigrams[(w1, w2)] = int(count)
        return bigrams

    @functools.lru_cache(maxsize=None)
    def edits1(self, word):
        """Return the set of words that are one edit away from 'word'."""
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def known(self, words):
        """Filter the set to only include words present in our corpus."""
        return {w for w in words if w in self.unigram_counts}

    @functools.lru_cache(maxsize=None)
    def candidates(self, word):
        """Generate possible spelling corrections for a word."""
        # If the word is known, return it immediately.
        if word in self.unigram_counts:
            return {word}
        # Otherwise, try words that are one edit away.
        edits = self.edits1(word)
        known_edits = self.known(edits)
        return known_edits if known_edits else {word}
    
    def unigram_probability(self, word):
        """Return the probability of a word based on unigram counts."""
        return self.word_probs.get(word, 1e-6)
    
    def bigram_probability(self, prev, word):
        """
        Compute the conditional probability P(word | prev) using bigram counts.
        Fallback to unigram probability if bigram data is missing.
        """
        if self.bigram_counts:
            total_prev = self.bigram_totals.get(prev, 0)
            if total_prev > 0:
                return self.bigram_counts.get((prev, word), 0) / total_prev
        return self.unigram_probability(word)
    
    def correct_sentence(self, sentence):
        """Correct a sentence using context-sensitive bigram probabilities."""
        tokens = sentence.split()
        corrected = []
        for i, token in enumerate(tokens):
            token_lower = token.lower()
            candidate_set = self.candidates(token_lower)
            candidates_list = list(candidate_set)
            
            # Use numpy to compute probabilities for candidates.
            if i == 0:
                probs = np.array([self.unigram_probability(c) for c in candidates_list])
            else:
                prev_word = corrected[i - 1]
                probs = np.array([self.bigram_probability(prev_word, c) for c in candidates_list])
                
            best_idx = np.argmax(probs)
            best = candidates_list[best_idx]
            corrected.append(best)
        return " ".join(corrected)

# Example usage:
corpus_file = "big.txt"        # Norvig's corpus (downloaded separately)
bigram_file = "bigrams.txt"    # Preprocessed bigram data file

# Initialize the spell corrector.
corrector = ContextSensitiveSpellCorrector(corpus_file, bigram_file)

In [25]:

# Test sentences: expected "doing sport" and "dying species"
test_sentences = [
	"dking sport",
	"dking species",
	'speling'
]

for sent in test_sentences:
	corrected = corrector.correct_sentence(sent)
	print(f"Original: {sent}\nCorrected: {corrected}\n")

Original: dking sport
Corrected: king sport

Original: dking species
Corrected: king species

Original: speling
Corrected: spelling



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

**Corpus and N-gram Data**

Unigram Dataset:
>Large unigram corpus (like Norvig’s “big.txt”) is loaded to build word frequency data. This helps rank correction candidates, favoring common words.

Bigram Dataset (Context Sensitivity):
>If available, we also load bigram frequencies. This allows us to calculate conditional probabilities \(P(\text{{word}} \mid \text{{prev}})\), helping the corrector pick words that fit the context, not just the spelling.


**Candidate Generation and Edit Distance**

Edit Distance 1 Only:
>Candidates are generated using the `edits1` function, which finds words one edit away (insertions, deletions, transpositions, replacements). This captures most common typos while keeping computation fast.

No Edit Distance 2:
>I skip edit distance 2 because it multiplies candidates and slows processing. Instead, algorithm rely on context or treat such words as unknown.

**Probability Weights and Smoothing**

Unigram and Bigram Probabilities:
>Corrector score candidates using unigram or bigram probabilities. If a word isn’t in the corpus, we assign a small default probability (1e-6) to avoid zero values.

Implicit Error Weighting: 
>Corrector don’t assign different weights to edit types. Limiting candidates to one edit naturally favors simpler, more likely corrections.

**Efficiency and Caching**

LRU Caching:
>I use Python’s `functools.lru_cache` for `edits1` and `candidates` to avoid recalculating results for the same word. This speeds up processing, especially for long texts.

NumPy for Probability Calculation:
>Probabilities are converted to NumPy arrays for faster, vectorized calculations.

**Sentence Correction Process**

Greedy Selection:  
>For each word, corrector pick the candidate with the highest unigram or bigram probability. While beam search could improve coherence, I chose greedy selection for speed and simplicity.

Case Normalization:
>Code convert tokens to lowercase before generating candidates. This avoids issues from capitalization mismatches.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [56]:
###CONSTANTS 
corpus_file = "big.txt" 
bigram_file = "bigrams.txt"
# Define a list of test sentences (assumed to be lower-case).
test_sentences = [
    "this is a smple test sentnce.",
    "the quick brown fox jumpd over the lazy dog.",
    "we are experimting with a spel corrector.",
    "please corect any mispelled words in this sntence.",
    "anothr exampl of a sentence with erors.",
    "it is importnt to have accurate spell chcking.",
    "sometimes computers mispell words too.",
    "the languge model must be robust and fast.",
    "she enjoys readng books and writng stories.",
    "he went to the stroe to by some groceries.",
    "their are many opportnities in the tech industrty.",
    "we will evaluat the performance of our spell checker.",
    "the algoritm behind the spell correction is critcal.",
    "ensuring proper grammer is essential for clarity.",
    "he recived a package in the mail yestreday.",
    "the managment team approved the new plan.",
    "it is necassary to proofred documents carefully.",
    "a good speling corrector can improove writting quality.",
    "our systm detected several errors in the text.",
    "this exampe is purposely mistaked for evaluation.",
    "the weather today is unusually cold and windy.",
    "the study of computr science involves logc and maths.",
    "programming requires precission and patence.",
    "the library offers a wide rang of books and articles.",
    "students are encourged to review their work.",
    "he always taks notes during lecture.",
    "the reserch paper was peer reviwed thrughly.",
    "an effective spell chcker needs a large corpus.",
    "the output of the model is quite impresive.",
    "we are curently working on improving the systm.",
    "the commitee met to discss the new proposal.",
    "the developrs are integrating new features into the app.",
    "this sentnce has multiple erors that need corection.",
    "accuracy in speling is essential for professionalism.",
    "she recieves many compliments on her writting.",
    "the officl meeting has been rescheduled for tuesday.",
    "we need to update the databse with new entries.",
    "the softwere developer fixed the bug in the code.",
    "this exampl illustrates the issues in text processing.",
    "we must implemnt a robust spell chekcer algorithm.",
    "the user interfac is clear and intuitive.",
    "analyzing large amounts of data is a challnge.",
    "the currnt version of the program contains bugs.",
    "it is important to maintain consistncy in data.",
    "the report was submitted by the dedicted team.",
    "the attendence at the conference was impressive.",
    "the opertion was completed within the time limit.",
    "new technologies are emerging fast in the industry.",
    "she has a keen eye for detail and critcal thinking.",
    "the experience was both enriching and challengng.",
    "improving our speling corrector will benefit many users.",
    "the datasheet contains in-depth informations about the product.",
    "the system performnce is evaluated under various conditions.",
    "he practised his writng skills daily.",
    "the course covers a wide range of computtion topics.",
    "a welldone spell checker can save time and reduce errors.",
    "the experiment involves both qualitative and quantitive analysis.",
    "her respons was quick and helpful.",
    "the survey results were summarised in the report.",
    "this sentnce intentionally incorrrect to test the system.",
    "the language model was trained on a verity of data sources.",
    "spell chekcing is a useful tool in many applications.",
    "the improvement in performance is noticable.",
    "our team consist of experinced professionals.",
    "a detailed analysis was conductd for this study.",
    "the text corpus provides a rich source of data.",
    "the webscraper gathered a large number of pages.",
    "each page was processed and split into sentnces.",
    "the output list of sentnces will be used for evaluation.",
    "the algorithm rturns the most probable corrections.",
    "the sample input contains several mispeled words.",
    "we use statistical models to rank the canddiates.",
    "the correction procedure takes context into acount.",
    "the system architecture was designed for scalabilty.",
    "machine learning can assist in spell correction.",
    "the evaluation metric includes both accuracy and recall.",
    "this test data simulates real-world mistkes.",
    "it is crucial to have a robust preprocessng pipeline.",
    "cleaning the text removes unwanted characters and tags.",
    "the iterative parsing helps manage memory effectively.",
    "our research focusses on improving error detection.",
    "the training corpus consists of millions of words.",
    "the implementtion of the algorithm was sucessful.",
    "the performance of the spell checker is evaluatd using benchmark tests.",
    "the results indicate a signficant improvement over previous versions.",
    "the datacleaning process is essential before analysis.",
    "the printed report contains various statstics and graphs.",
    "the programm was developed using python and xml libraries.",
    "each sample sentence has been curately selected for testing.",
    "the text has been normalized to lower case and cleaned thoroughly.",
    "the evaluation process is iterative and data-driven.",
    "user feedback is important to refine the correction algorithm.",
    "the model's prediction is compared against a gold standard.",
    "the sentence segmentation is carried out using nltk.",
    "the python code integrates download, preprocessing, and extraction.",
    "the ultimate goal is to achieve high accuracy in spell correcton.",
    "the test corpus is diverse and covers many topics.",
    "the refined algorithm shows promise in real-world applications.",
    "the output of the preprocessing pipeline is stored in a list.",
    "this final sentence completes our 100 evaluation samples."
]


noise_probs = [0.1, 0.3, 0.5]


In [57]:
test_sentences

['this is a smple test sentnce.',
 'the quick brown fox jumpd over the lazy dog.',
 'we are experimting with a spel corrector.',
 'please corect any mispelled words in this sntence.',
 'anothr exampl of a sentence with erors.',
 'it is importnt to have accurate spell chcking.',
 'sometimes computers mispell words too.',
 'the languge model must be robust and fast.',
 'she enjoys readng books and writng stories.',
 'he went to the stroe to by some groceries.',
 'their are many opportnities in the tech industrty.',
 'we will evaluat the performance of our spell checker.',
 'the algoritm behind the spell correction is critcal.',
 'ensuring proper grammer is essential for clarity.',
 'he recived a package in the mail yestreday.',
 'the managment team approved the new plan.',
 'it is necassary to proofred documents carefully.',
 'a good speling corrector can improove writting quality.',
 'our systm detected several errors in the text.',
 'this exampe is purposely mistaked for evaluation.',


In [58]:
#### CODE FROM NORVIG IMPLEMENTATION

def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open(corpus_file).read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [59]:
class NorvigFacade:
    def __init__(self):
        pass

    def correct_sentence(self, sentence):
        "Correct each word in the sentence using the Norvig approach."
        return " ".join(correction(word) for word in sentence.split())

norvig_corrector = NorvigFacade()

In [60]:
def add_noise_to_word(word, noise_prob):
    """With probability noise_prob, perform one random edit on the word."""
    if random.random() < noise_prob and word:
        operations = ['delete', 'insert', 'substitute', 'transpose']
        op = random.choice(operations)
        letters = 'abcdefghijklmnopqrstuvwxyz'
        if op == 'delete' and len(word) > 0:
            idx = random.randint(0, len(word)-1)
            return word[:idx] + word[idx+1:]
        elif op == 'insert':
            idx = random.randint(0, len(word))
            return word[:idx] + random.choice(letters) + word[idx:]
        elif op == 'substitute' and len(word) > 0:
            idx = random.randint(0, len(word)-1)
            return word[:idx] + random.choice(letters) + word[idx+1:]
        elif op == 'transpose' and len(word) > 1:
            idx = random.randint(0, len(word)-2)
            return word[:idx] + word[idx+1] + word[idx] + word[idx+2:]
    return word

def add_noise(sentence, noise_prob):
    """Apply noise to every word in the sentence with given probability."""
    words = sentence.split()
    noisy_words = [add_noise_to_word(word, noise_prob) for word in words]
    return " ".join(noisy_words)

def evaluate(corrector, test_sentences, noise_prob):
    total_words = 0
    correct_words = 0
    total_sentences = 0
    correct_sentences = 0
    
    for sentence in test_sentences:
        noisy_sentence = add_noise(sentence, noise_prob)
        corrected = corrector.correct_sentence(noisy_sentence)
        
        orig_words = sentence.split()
        corr_words = corrected.split()
        total_words += len(orig_words)
        # For word-level, count matching words (compare up to the length of the original)
        correct_words += sum(1 for o, c in zip(orig_words, corr_words) if o == c)
        
        total_sentences += 1
        if orig_words == corr_words:
            correct_sentences += 1
    word_accuracy = correct_words / total_words
    sentence_accuracy = correct_sentences / total_sentences
    return word_accuracy, sentence_accuracy

cs_corrector = ContextSensitiveSpellCorrector(corpus_file, bigram_file)

In [61]:

print("Evaluation of Spell Correctors on Noisy Test Set:\n")
print("{:<12}{:<40}{:<40}".format("Noise Prob", "Context-Sensitive (Word / Sentence)", "Norvig (Word / Sentence)"))
for noise_prob in noise_probs:
    cs_word_acc, cs_sent_acc = evaluate(cs_corrector, test_sentences, noise_prob)
    norvig_word_acc, norvig_sent_acc = evaluate(norvig_corrector, test_sentences, noise_prob)
    print("{:<12.2f}{:<20.2%} / {:<17.2%}{:<20.2%} / {:<17.2%}".format(
        noise_prob, cs_word_acc, cs_sent_acc, norvig_word_acc, norvig_sent_acc))

Evaluation of Spell Correctors on Noisy Test Set:

Noise Prob  Context-Sensitive (Word / Sentence)     Norvig (Word / Sentence)                
0.10        78.14%               / 7.00%            72.86%               / 2.00%            
0.30        73.37%               / 8.00%            64.57%               / 1.00%            
0.50        66.33%               / 1.00%            60.30%               / 1.00%            


my implementaion has a little bit higher metrics than naive implementsion

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)